In [1]:
import os
import sys
import time
import datetime
import argparse
import pickle
from skimage.data import imread
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torch.autograd import Variable
import torch.optim as optim

sys.path.append('/floyd/home/PyTorch-YOLOv3_lite')

In [2]:
%load_ext autoreload
%autoreload 2
from models import *
from utils.utils import *
from utils.datasets import *
from utils.parse_config import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
args = ['--image_folder','/floyd/input/airbus/train',
        '--box_file','/floyd/input/airbus/box_data.pickle',
        '--model_config_path','/floyd/home/PyTorch-YOLOv3_lite/config/classifier.cfg',
        '--weights_path','/floyd/input/weights/6.weights',
        '--checkpoint_dir','/floyd/home/output',
        '--img_size','64',
        '--epochs','30',
        '--n_cpu','4',
        '--use_cuda','True']
parser = argparse.ArgumentParser()
parser.add_argument('--epochs', type=int, default=30, help='number of epochs')
parser.add_argument('--image_folder', type=str, default='data/samples', help='path to dataset')
parser.add_argument('--box_file', type=str, default='data/samples', help='path to box data')
parser.add_argument('--batch_size', type=int, default=16, help='size of each image batch')
parser.add_argument('--model_config_path', type=str, default='config/yolov3.cfg', help='path to model config file')
parser.add_argument('--weights_path', type=str, default='weights/yolov3.weights', help='path to weights file')
parser.add_argument('--conf_thres', type=float, default=0.8, help='object confidence threshold')
parser.add_argument('--nms_thres', type=float, default=0.4, help='iou thresshold for non-maximum suppression')
parser.add_argument('--n_cpu', type=int, default=0, help='number of cpu threads to use during batch generation')
parser.add_argument('--img_size', type=int, default=416, help='size of each image dimension')
parser.add_argument('--checkpoint_interval', type=int, default=1, help='interval between saving model weights')
parser.add_argument('--checkpoint_dir', type=str, default='checkpoints', help='directory where model checkpoints are saved')
parser.add_argument('--use_cuda', type=bool, default=True, help='whether to use cuda if available')
opt = parser.parse_args(args)

In [13]:
cuda = torch.cuda.is_available() and opt.use_cuda

os.makedirs(opt.checkpoint_dir, exist_ok=True)

# Get hyper parameters
hyperparams     = parse_model_config(opt.model_config_path)[0]
learning_rate   = float(hyperparams['learning_rate'])
momentum        = float(hyperparams['momentum'])
decay           = float(hyperparams['decay'])
burn_in         = int(hyperparams['burn_in'])

# Initiate model
model = Darknet_Classifier(opt.model_config_path)
model.load_weights(opt.weights_path)
model.apply(weights_init_normal)

if cuda:
    model = model.cuda()

model.train();



In [14]:
# Get dataloader
with open(opt.box_file, 'rb') as f:
    data = pickle.load(f)
    
dataloader = torch.utils.data.DataLoader(
    BoxedDataset(opt.image_folder,(768,768),data),
    batch_size=opt.batch_size, shuffle=True, num_workers=opt.n_cpu)

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum, dampening=0, weight_decay=decay)


In [15]:
class BoxDetections(Dataset):
    def __init__(self,imgdir,imgsize,data):
        super().__init__()
        self.imgdir = imgdir
        self.imgsize = imgsize
        self.box_image_ids = data['box_image_ids']
        self.L1 = data['L1']
        self.L2 = data['L2']
        self.R0 = data['C0'] # accidently switched when saved
        self.C0 = data['R0'] # accidently switched when saved
        self.theta = data['theta']
        self.all_image_ids = data['all_image_ids']
        self.num_ships = data['num_ships']
        self.total_ships = sum(self.num_ships)
        self.total_imgs = len(self.all_image_ids)
    def __len__(self):
        return self.total_imgs
    def __getitem__(self,idx):
        img_path = os.path.join(self.imgdir,self.all_image_ids[idx])
        img = np.array(Image.open(img_path)) 
        # Channels-first
        img = np.transpose(img, (2, 0, 1))
        # As pytorch tensor
        img = torch.from_numpy(img).float()
        # Find boxes in img
        labels = None
        if self.num_ships[idx] != 0:
            box_inds = [i for i,x in enumerate(self.box_image_ids) if x is self.all_image_ids[idx]]
            labels = np.loadtxt(label_path).reshape(-1, 5)
            # Extract coordinates for unpadded + unscaled image
            x1 = w * (labels[:, 1] - labels[:, 3]/2)
            y1 = h * (labels[:, 2] - labels[:, 4]/2)
            x2 = w * (labels[:, 1] + labels[:, 3]/2)
            y2 = h * (labels[:, 2] + labels[:, 4]/2)
            # Adjust for added padding
            x1 += pad[1][0]
            y1 += pad[0][0]
            x2 += pad[1][0]
            y2 += pad[0][0]
            # Calculate ratios from coordinates
            labels[:, 1] = ((x1 + x2) / 2) / imgsize
            labels[:, 2] = ((y1 + y2) / 2) / imgsize
            labels[:, 3] *= w / imgsize
            labels[:, 4] *= h / imgsize
        # Fill matrix
        filled_labels = np.zeros((self.max_objects, 5))
        if labels is not None:
            filled_labels[range(len(labels))[:self.max_objects]] = labels[:self.max_objects]
        filled_labels = torch.from_numpy(filled_labels)
        return img_path, img, filled_labels
    

In [16]:
bdtest = BoxDetections(opt.image_folder,(768,768),data)

NameError: name 'input_img' is not defined

In [33]:
box_inds = [i for i,x in enumerate(bdtest.box_image_ids) if x is bdtest.all_image_ids[11]]

In [34]:
box_inds

[]